### Import libraries

In [ ]:
from libs.Dataset import TripletTrashbinDataModule
from libs.Model import TripletNetworkTask, evaluating_performance
from torch import nn
from torchvision.models import squeezenet1_1
import torch
import pytorch_lightning as pl
from torchvision.models import squeezenet1_1
from torch import nn
import torch
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import progress
from torchvision.models import squeezenet1_1
from torch import nn
import torch
# or to ignore all warnings that could be false positives
import warnings
from pytorch_lightning.utilities.warnings import PossibleUserWarning
warnings.filterwarnings("ignore", category=PossibleUserWarning)

In [ ]:
data_dir = "dataset/"
path_gdrive = ""
device = "cpu"

class_dict = {
    0: 'empty',
    1: 'half',
    2: 'full'
}

data_img_size = 224
data_batch_size = 256
# otherwise, return warning
n_workers = 0  # os.cpu_count()

Load del dataModule

In [ ]:
dm = TripletTrashbinDataModule(img_size=data_img_size,num_workers=n_workers)
# dm.prepare_data()
dm.setup()

### Pre-execution

In [ ]:
squeezeNet = squeezenet1_1(pretrained=True)
squeezeNet.classifier = nn.Identity()

Dimensione delle feature richieste dal modello <b>TODO: scrivi meglio</b>

In [ ]:
squeezeNet(torch.zeros(1,3,data_img_size,data_img_size)).shape

In [ ]:
triplet_squeezeNet = TripletNetworkTask(squeezeNet, lr=0.002, batch_size=data_batch_size) # parto dal LR trovato nel precedente studio

Utilizzando le funzioni di libreria di pytorch_lighting controllo la dimensione del batch <b>SCRIVI MEGLIO</b>

In [ ]:
trainer = pl.Trainer(auto_scale_batch_size="power", max_epochs=-1)
trainer.tune(triplet_squeezeNet, datamodule=dm)

Utilizzando la funzione di libreria di pytorch lighting vedo qual è il migliore LR <b>SCRIVI MEGLIO</b>

In [ ]:
trainer = pl.Trainer(auto_lr_find=True)
lr_finder = trainer.tune(triplet_squeezeNet, datamodule=dm)
# TODO: trova il modo di fare il grafico del LR trovato!
# triplet_mobileNet.hparams.lr = new_lr # TODO: da verificare se si deve fare così o col costruttore
# TODO: SOStituisci il LR
lr_finder


### Training con loss A

Valuto le performance prima del training

In [ ]:
# TODO: Instanzio nuovamente il TripletNetworkTask con batch_size e lr in base al risultato di sopra ^ 

In [ ]:
evaluating_performance(triplet_squeezeNet, dm, data_batch_size)

Alleno il modello per 10 epoche con loss A

In [ ]:
logger = TensorBoardLogger("models/logs", name="triplet_squeezeNet_v1")

trainer = pl.Trainer(gpus=0,
                    max_epochs=10,
                    callbacks=[progress.TQDMProgressBar()],
                    logger=logger,
                    accelerator="auto",
                    )

trainer.fit(model=triplet_squeezeNet, datamodule=dm)
trainer.save_checkpoint('models/triplet_squeezeNet_v1_{}_batch.ckpt'.format(data_batch_size) )
torch.save(trainer.model.state_dict(), "models/triplet_squeezeNet_v1_{}_batch.pth")


Valuto le performance ottenute

In [ ]:
evaluating_performance(triplet_squeezeNet, dm, data_batch_size)

#### Valuto con B

Devo fare di nuovo l'esecuzione del batch e del lr con V2 ????? SI!